### Species Widget

### Covert into 3 tables:
#### orinal matrix data :
location_id: string  
species_scientific_names {string, n columns based on species list}: boolean   {represents presence/abscence}
#### Table 1:
location_id: string  
species: array[species_id]
#### Table 2
species_id: number  
scientific_name: str  
common_name: str  
iucn_url: url_str  
red_list_cat: enum {ex, ew, re, cr, en, vu, lr,nt,lc,dd} - reference Red list categories

In [9]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona
import requests

/Users/tamarahuete/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
sp = pd.read_csv('../../data/Mangrove_Biodiversity.csv', encoding = 'latin-1')

In [11]:
sp.head()

,Species,Acanthus ebracteatus,Acanthus ilicifolius,Acrostichum aureum,Acrostichum danaeifolium,Acrostichum speciosum,Aegialitis annulata,Aegialitis rotundifolia,Aegiceras corniculatum,Aegiceras floridum,...,Sonneratia caseolaris,Sonneratia griffithii,Sonneratia lanceolata,Sonneratia ovata,Sonneratia x gulngai,Sonneratia x hainanensis,Sonneratia x urama,Tabebuia palustris,Xylocarpus granatum,Xylocarpus moluccensis
0,Comoros,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kenya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Madagascar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Mauritius,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mayotte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Table 2:
* First we need to retrieve the needed data from the IUCN API.  
We can perform a search on IUCN main page (https://www.iucnredlist.org/),for example, to retrieve the first species Acanthus ebracteatus.
That will send us to another page: https://www.iucnredlist.org/search?query=Acanthus%20ebracteatus&searchType=species
If we open the inspector tab we can see the search querys from 'Network'
We can click on one of the 'search?size...' and look at the headers and payload. Not all querys return the same information, we need t find one that is returning the common name, species id, iucn status, etc.
Once we find it, we can do right click --> Copy --> Copy as cURL. 
The curl command is shown below. We can reformat this in python format using https://curlconverter.com/#python  
We can make this request simpler by eliminating unecessary params and we can make it generic by passing the species name to the query term.

* Once we have all the data we can add it to a dataframe

* What to do with hibrid species??? they don't have an IUCN species identifier
For now I am adding 'hibrid{list_number}' but we should think about how to identify them

In [ ]:
curl 'https://www.iucnredlist.org/dosearch/assessments/_search?size=60&_source=false&from=0&track_total_hits=true' \
  -H 'authority: www.iucnredlist.org' \
  -H 'pragma: no-cache' \
  -H 'cache-control: no-cache' \
  -H 'sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"' \
  -H 'dnt: 1' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36' \
  -H 'sec-ch-ua-platform: "Linux"' \
  -H 'content-type: application/json' \
  -H 'accept: */*' \
  -H 'origin: https://www.iucnredlist.org' \
  -H 'sec-fetch-site: same-origin' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-dest: empty' \
  -H 'referer: https://www.iucnredlist.org/search?query=canis%20lupus&searchType=species' \
  -H 'accept-language: en,es;q=0.9,gl;q=0.8' \
  -H 'cookie: _ga=GA1.2.1721324640.1640163956; _gid=GA1.2.729186747.1640163956; _application_devise_session=U2NyVC9NaS8xOXdEeEdFOFprY0VWeVZTWE0vWWZZb2NQNEJyRlNlejVXK2Y1N1lOcUQwRDlkbUd0OXk0MmEvZS84RFVPQUtGNEJUenZQdXVQZzVqblkrT050RC9kMmJCTGtESTh5c0JzVWF1MzRKRkV0cXJPWkFhUlZIRXlBZWI1YVFwUDIrRXA0MHR2RVNnNEFIb1JGMUNteS9HUFhid05FZjI2WVpzSVorQjRlVmNIQjd1eWc5RlArR09lRjFGbGQ1ZUJFSlJoN201N1JQT0U2T3hLN2lERkdjeE00OTBIaERjN09ocG9KdTNSMFpnbTE2a3RVWElQR3Q5cjlaby0tUEMvSHlIMnBJckludlhEZjRvY0RRUT09--f221903eb125a5b45ea7b6dc7d9148fa0643c650' \
  --data-raw '{"stored_fields":["hasImage","hasPoints","hasRanges","image.id","image.url","image.urlThumb","image.credit","scopes.id","scopes.code","scopes.jsonDescription","kingdomName","className","commonName","scientificName","sisTaxonId","redListCategory.scaleCode","redListCategory.order","redListCategory.code","redListCategory.jsonDescription","populationTrend.id","populationTrend.code","populationTrend.jsonDescription","hasGreen","greenListCategory.scaleCode","greenListCategory.name"],"query":{"bool":{"must":[{"multi_match":{"query":"canis lupus","type":"phrase_prefix","fields":["commonName^12","commonNames^10","scientificName^8","keywords^4","synonyms^2","assessors","sisTaxonId","id"],"lenient":true,"max_expansions":100}}],"filter":{"bool":{"filter":[{"terms":{"scopes.code":["1"]}},{"terms":{"taxonLevel":["Species"]}}],"should":[],"minimum_should_match":0}},"should":[{"term":{"hasImage":{"value":true,"boost":6}}}]}},"sort":[{"_score":{"order":"desc"}}]}' \
  --compressed

In [12]:
sp_list= sp.columns[1:]

In [40]:
#format species name
sp_str =sp_list[2].lower()#.replace(" ", "%20")

In [45]:
def getIUCNdata(species):
    sp_str =species.lower()

    headers = {
        'authority': 'www.iucnredlist.org',
        'content-type': 'application/json'}

    data = json.dumps({"stored_fields":["hasImage","hasPoints","hasRanges","image.id","image.url",\
                "image.urlThumb","image.credit","scopes.id","scopes.code","scopes.jsonDescription",\
                "kingdomName","className","commonName","scientificName","sisTaxonId",\
                "redListCategory.scaleCode","redListCategory.order","redListCategory.code",\
                "redListCategory.jsonDescription","populationTrend.id","populationTrend.code",\
                "populationTrend.jsonDescription","hasGreen","greenListCategory.scaleCode",\
                "greenListCategory.name"],\
                "query":{"bool":{"must":[{"multi_match":\
                {"query":f"{sp_str}","type":"phrase_prefix",\
                "fields":["commonName^12","commonNames^10",\
                "scientificName^8","keywords^4","synonyms^2",\
                "assessors","sisTaxonId","id"],"lenient":True,\
                "max_expansions":100}}],\
                "filter":{"bool":{"filter":[{"terms":{"scopes.code":["1"]}},\
                {"terms":{"taxonLevel":["Species"]}}],"should":[],"minimum_should_match":0}},\
                "should":[{"term":{"hasImage":{"value":True,"boost":6}}}]}},"sort":[{"_score":{"order":"desc"}}]})

    response = requests.get('https://www.iucnredlist.org/dosearch/assessments/_search', headers=headers, data=data)
    return response.json()

In [123]:
records ={'species_id':[],
         'scientific_name':[],
         'common_name':[],
         'iucn_url':[],
         'red_list_cat':[]}
for n,species in enumerate(sp_list):
    out = getIUCNdata(species)
    records['scientific_name'].append(species)
    records['iucn_url'].append(f'https://www.iucnredlist.org/search?query={species.replace(" ","%20")}&searchType=species')
    
    ### Sepcies id
    try:
        records['species_id'].append(out['hits']['hits'][0]['_id'])
    except:
        records['species_id'].append(f'hibrid{n}')
    
    ### Common name
    try:
        if 'commonName' in out['hits']['hits'][0]['fields']:
            records['common_name'].append(out['hits']['hits'][0]['fields']['commonName'][0])
        else:
            records['common_name'].append(out['hits']['hits'][1]['fields']['commonName'][0])
    except:
        records['common_name'].append(np.nan)
    
    ### IUCN category
    try:
        records['red_list_cat'].append(out['hits']['hits'][0]['fields']['redListCategory.scaleCode'][0])
    except:
        records['red_list_cat'].append(np.nan)
        

In [124]:
table2 = pd.DataFrame(data=records)

In [125]:
table2

,species_id,scientific_name,common_name,iucn_url,red_list_cat
0,7621003,Acanthus ebracteatus,Holy Mangrove,https://www.iucnredlist.org/search?query=Acant...,lc
1,6536949,Acanthus ilicifolius,Holy Mangrove,https://www.iucnredlist.org/search?query=Acant...,lc
2,7366131,Acrostichum aureum,NaN,https://www.iucnredlist.org/search?query=Acros...,lc
3,7613396,Acrostichum danaeifolium,NaN,https://www.iucnredlist.org/search?query=Acros...,lc
4,7614751,Acrostichum speciosum,NaN,https://www.iucnredlist.org/search?query=Acros...,lc
...,...,...,...,...,...
68,hibrid68,Sonneratia x hainanensis,NaN,https://www.iucnredlist.org/search?query=Sonne...,NaN
69,hibrid69,Sonneratia x urama,NaN,https://www.iucnredlist.org/search?query=Sonne...,NaN
70,7610513,Tabebuia palustris,NaN,https://www.iucnredlist.org/search?query=Tabeb...,vu
71,7624881,Xylocarpus granatum,NaN,https://www.iucnredlist.org/search?query=Xyloc...,lc


In [ ]:
### Table 1
Convert Species names to identifiers

In [140]:
#sp.stack().reset_index().set_axis('From To Distance'.split(), axis=1)

In [153]:
table1= pd.DataFrame(data = {'location_id':sp.Species,'species':np.nan})
for country in sp.Species:
    df = sp[sp['Species']==country]
    for col in df.columns:
        if df[[col]].values[0]==0:
            df.drop(col,inplace=True,axis=1)
            sp_names = df.columns
    table1.loc[table1['location_id']==country,'species']= str(list(table2[table2.scientific_name.isin(sp_names)].species_id))

/Users/tamarahuete/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [154]:
table1

,location_id,species
0,Comoros,"['7619457', '7610926', '7617531', '7627492', '..."
1,Kenya,"['7619457', '7610926', '7617531', '7627492', '..."
2,Madagascar,"['7619457', '7610926', '7617531', '7627492', '..."
3,Mauritius,"['7366131', '7610926', '7622565', '7618520']"
4,Mayotte,"['7619457', '7610926', '7617531', '7627492', '..."
...,...,...
121,Nigeria,"['7366131', '7613866', '7612125', '7609219', '..."
122,Sao.Tome.and.Principe,"['7366131', '7613866', '7612125', 'hibrid56']"
123,Senegal,"['7366131', '7613866', '7612125', '7609219', '..."
124,Sierra.Leone,"['7613866', '7612125', '7609219', '69024847', ..."
